In [ ]:
#门控单元（GRU）
#为了更好地捕捉时间序列中时间步距离较大的依赖关系
#重置门（reset gate）和更新门（update gate)
#输入均为当前时间步输入Xt与上一时间步隐藏状态Ht−1，输出由激活函数为sigmoid函数的全连接层计算得到
#重置门：Rt=σ(XtWxr+Ht−1Whr+br)
#更新门：Zt=σ(XtWxz+Ht−1Whz+bz),
#Xt ∈ R^{n×d}样本数为n 输入个数为d

#候选隐藏状态
#将当前时间步重置门的输出与上一时间步隐藏状态做按元素乘法（符号为⊙），
#然后，将按元素乘法的结果与当前时间步的输入连结，再通过含激活函数tanh的全连接层计算出候选隐藏状态，其所有元素的值域为[−1,1]
#候选隐藏状态：H~t=tanh(XtWxh+(Rt⊙Ht−1)Whh+bh),
#重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态。
#而上一时间步的隐藏状态可能包含了时间序列截至上一时间步的全部历史信息。因此，重置门可以用来丢弃与预测无关的历史信息。

#隐藏状态
#时间步t的隐藏状态Ht的计算使用当前时间步的更新门Zt来对上一时间步的隐藏状态Ht−1和当前时间步的候选隐藏状态H˜t做组合：
#Ht=Zt⊙Ht−1+(1−Zt)⊙H~t.
#    重置门有助于捕捉时间序列里短期的依赖关系；
#更新门有助于捕捉时间序列里长期的依赖关系。

In [1]:
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices,char_to_idx,idx_to_char,vocab_size) = d2l.load_data_jay_lyrics()

In [2]:
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
print('will use',device)
#初始化模型参数
def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    def _three():
        return (_one((num_inputs,num_hiddens)),
               _one((num_hiddens,num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True))
    W_xz,W_hz,b_z = _three()#更新门参数
    W_xr,W_hr,b_r = _three()#重置门参数
    W_xh,W_hh,b_h = _three()#候选参数隐藏状态
    
    #输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)
    return nn.ParameterList([W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q])

will use cuda


In [3]:
#定义模型
#定义隐藏状态初始化函数init_gru_state,返回由一个形状为(批量大小, 隐藏单元个数)的值为0的Tensor组成的元组。
def init_gru_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device), )
def gru(inputs,state,params):
    W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X,W_xz)+torch.matmul(H,W_hz)+b_z)
        R = torch.sigmoid(torch.matmul(X,W_xr)+torch.matmul(H,W_hr)+b_r)
        H_tilda = torch.tanh(torch.matmul(X,W_xh)+torch.matmul(R*H,W_hh)+b_h)
        H = Z*H + (1-Z)*H_tilda
        Y = torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

In [4]:
#相邻采样
num_epochs,num_steps,batch_size,lr,clipping_theta = 160,35,32,1e2,1e-2
pred_period,pred_len,prefixes = 40,50,['分开','不分开']
d2l.train_and_predict_rnn(gru,get_params,init_gru_state,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,
                         char_to_idx,False,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,
                         prefixes)

epoch 40, perplexity 153.617031, time 0.47 sec
 - 分开 我不的让我 我想你的让我 我想你的让我 爱爱我 我想你的让我 想想你的让我 我想你的让我 爱爱我 
 - 不分开 我想你的让我 我想你的让我 我想你的让我 爱爱我 我想你的让我 想想你的让我 我想你的让我 爱爱我
epoch 80, perplexity 32.499173, time 0.49 sec
 - 分开 我想要这样的怒火 像知开不不能 我不要再想 我不要再想 我不能再想 我不能再想 我不能再想 我不能
 - 不分开 我不要再想 我不要再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
epoch 120, perplexity 5.653013, time 0.50 sec
 - 分开我 泪你是一个人慢慢 我想想你 我不要 我不再再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏
 - 不分开 为是我怕见你是一场悲剧 我想我这辈子注定一个人演戏 就后开不起口 不要再这样我妈妈 我想你的微笑著
epoch 160, perplexity 1.681346, time 0.47 sec
 - 分开 让我的外前 这样在在身听 静静了在落落 我想能的睡笑每天都能看到  我知道这里很美但家乡的你更美走
 - 不分开 你是经不开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活


In [ ]:
#LSTM
#输入门（input gate）、遗忘门（forget gate）和输出门（output gate）
#输入门：It = σ(Xt​Wxi+Ht−1​Whi+bi)
#遗忘门：Ft = σ(Xt​Wxf+Ht−1​Whf+bf)
#输出门：Ot = σ(Xt​Wxf\o+Ht−1​Who+bo)
#Xt∈Rn×d(样本数n,输入个数为d)，Ht−1∈Rn×h，It、Ft、Ot都∈Rn×h

#候选记忆细胞
#C~t=tanh(Xt​Wxc+Ht−1​Whc+bc),

#记忆细胞
#当前时间步记忆细胞Ct∈Rn×h的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息，并通过遗忘门和输入门来控制信息的流动：
#Ct=Ft⊙Ct−1+It⊙C~t
#遗忘门控制上一时间步的记忆细胞Ct−1中的信息是否传递到当前时间步，
#而输入门则控制当前时间步的输入Xt通过候选记忆细胞C˜t如何流入当前时间步的记忆细胞。
#有了记忆细胞以后，接下来我们还可以通过输出门来控制从记忆细胞到隐藏状态Ht∈Rn×h的信息的流动：
#Ht=Ot⊙tanh(Ct).

In [5]:
#实现
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
print("will use",device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    def _three():
        return (_one((num_inputs,num_hiddens)),
               _one((num_hiddens,num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True))
    W_xi,W_hi,b_i = _three()
    W_xf,W_hf,b_f = _three()
    W_xo,W_ho,b_o = _three()
    W_xc,W_hc,b_c = _three() #候选记忆细胞参数
    
    #输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)
    return nn.ParameterList([W_xi,W_hi,b_i,W_xf,W_hf,b_f,W_xo,W_ho,b_o,W_xc,W_hc,b_c,W_hq,b_q])

will use cuda


In [11]:
#模型定义
def init_lstm_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),
           torch.zeros((batch_size,num_hiddens),device=device))
#只有隐藏状态会传递到输出层，而记忆细胞不参与输出层的计算
def lstm(inputs,state,params):
    [W_xi,W_hi,b_i,W_xf,W_hf,b_f,W_xo,W_ho,b_o,W_xc,W_hc,b_c,W_hq,b_q] = params
    (H,C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X,W_xi)+torch.matmul(H,W_hi)+b_i)
        F = torch.sigmoid(torch.matmul(X,W_xf)+torch.matmul(H,W_hf)+b_f)
        O = torch.sigmoid(torch.matmul(X,W_xo)+torch.matmul(H,W_ho)+b_o)
        C_tilda = torch.tanh(torch.matmul(X,W_xc)+torch.matmul(H,W_hc)+b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,C)

In [12]:
num_epochs,num_steps,batch_size,lr,clipping_theta = 160,35,32,1e2,1e-2
pred_period,pred_len,prefixes = 40,50,['分开','不分开']
d2l.train_and_predict_rnn(lstm,get_params,init_lstm_state,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,
                         char_to_idx,False,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,
                         prefixes)

epoch 40, perplexity 216.616711, time 0.57 sec
 - 分开 我不的我 我不的 我不 我不的 我不 我不的 我不 我不的 我不 我不的 我不 我不的 我不 我不
 - 不分开 我不的我 我不的 我不 我不的 我不 我不的 我不 我不的 我不 我不的 我不 我不的 我不 我不
epoch 80, perplexity 78.455674, time 0.57 sec
 - 分开 我想想你的你我 想想 我想你的可爱我 别不的让我想要的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让
 - 不分开 我想你的爱我 我想 我想想我 我不不觉 我不不觉 我不不觉 我不不觉 我不不觉 我不不觉 我不不觉
epoch 120, perplexity 18.749026, time 0.62 sec
 - 分开 我想你的太笑 有后后人 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快
 - 不分开你 想想你的你笑我 别发我 你怪我 说你怎么 我想就这样着你的手不放开 爱可不可不简简单 没有我没你
epoch 160, perplexity 4.453000, time 0.62 sec
 - 分开 我想带你 我不要这是头  后后你在我有多 甩怪我 别怪我 说你怎么停留 一直在秋留 我该好这生活 
 - 不分开你 想要到你 我跟一定节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 静知悄悄默


In [16]:
#简洁实现
lr = 1e-2
lstm_layer = nn.LSTM(input_size=vocab_size,hidden_size=num_hiddens)
model = d2l.RNNModel(lstm_layer,vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.026910, time 0.04 sec
 - 分开 你说 苦笑常常陪着你 在一起有点勉强 该不该现在休了我 不想太多 我想一定是我听错弄错搞错 拜托 
 - 不分开 你说 苦笑常常陪着你 在一起有点勉强 该不该现在休了我 不想太多 我想一定是我听错弄错搞错 拜托 
epoch 80, perplexity 1.032403, time 0.03 sec
 - 分开 我才送到 你叫我的爸 你当榜样  好多的假像 妈妈常说乖听你爸的话  你叫我怎么跟你像 不要再这样
 - 不分开 你说 苦笑常常陪着你 在一起有点勉强 该不该现在休了我 不想太多 我想一定是我听错弄错搞错 拜托 
epoch 120, perplexity 1.015916, time 0.03 sec
 - 分开 我才的美 沙漠之中怎么会有泥鳅 话说完飞过一只海鸥 大峡谷的风呼啸而过 是谁说没有 有一条热昏头的
 - 不分开 我才送到你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温
epoch 160, perplexity 1.019827, time 0.04 sec
 - 分开 你说啊 你怎么打我手 你说啊 是不是你不想活 说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很久
 - 不分开不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 开不了口 


In [ ]:
#双向循环神经网络通过增加从后往前传递信息的隐藏层来更灵活地处理这类信息。
#时间步正向隐藏状态为H−→t∈Rn×h Ht∈Rn×h（正向隐藏单元个数为hh）， 
#反向隐藏状态为H←−t∈Rn×h Ht∈Rn×h（反向隐藏单元个数为hh）
#连结两个方向的隐藏状态H−→t和H←−t来得到隐藏状态Ht∈Rn×2h，并将其输入到输出层。输出层计算输出Ot